# Convert Jama Glossary to LaTeX

Step 1. In Jama, go to the glossary and choose *Export* $\to$ *Excel* to write out `CTA-Glossary.xls`

Step 2. read it into a Pandas DataFrame:

Note that the header is on row 3, so we need to specify that (the rows before that will be ignored)

In [147]:
import pandas as pd
import numpy as np

In [99]:
glossary = pd.read_excel(
    "CTA-Glossary.xls", 
    header=3, 
    sheet_name='Sheet1',
    usecols=[0,1,2,3,4,5],
    converters={'ID': lambda x: str(x)}
)  

In [100]:
glossary.head()

,Modified Date,Last Activity Date,Name,Description,ID,Status
0,09/06/2017,09/06/2017,CTA Constituents,NaN,NaN,NaN
1,16/05/2018,16/05/2018,CTAO,"The Cherenkov Telescope Array Observatory, an ...",NaN,Stable
2,16/05/2018,16/05/2018,CTA North,CTA Observation site hosting an Array of Chere...,NaN,Stable
3,16/05/2018,16/05/2018,CTA South,CTA Observation site hosting an Array of Chere...,NaN,Stable
4,16/05/2018,16/05/2018,Headquarters,"The primary centre for CTAO governance, admini...",NaN,Stable


In [142]:
glossary = glossary.dropna(subset=['Description']) # get rid of undefined terms

In [143]:
glossary_rec = "\\newglossaryentry{{{name}}}{{name={{{name}}}, description={{{description}}}}}\n\n"

In [144]:
print(glossary_rec.format(name="thingy", description="stuff"))

\newglossaryentry{thingy}{name={thingy}, description={stuff}}




In [159]:
with open("cta-glossary-defs.inc", 'w') as outfile:
    for name, description in zip(glossary.Name, glossary.Description):
        outfile.write(
            glossary_rec.format(
                name=str(name).strip(), 
                description=str(description).strip()
            )
        )


In [161]:
! head cta-glossary-defs.inc

\newglossaryentry{CTAO}{name={CTAO}, description={The Cherenkov Telescope Array Observatory, an international user facility distributed over four primary sites: a Headquarters, Science Data Management Centre, and two array-sites in the norhtern and southern hemispheres: CTA-N and CTA-S.}}

\newglossaryentry{CTA North}{name={CTA North}, description={CTA Observation site hosting an Array of Cherenkov Telescopes, in the Northern Hemisphere. The selected CTA North site is Observatorio del Roque de los Muchachos (ORM), La Palma, Spain. CTA North may be abbreviated as CTA-N.}}

\newglossaryentry{CTA South}{name={CTA South}, description={CTA Observation site hosting an Array of Cherenkov Telescopes, in the Southern Hemisphere. The selected CTA South site is lies between the locations of the Paranal and Armazones ESO telescope sites in Chile. CTA South may be abbreviated as CTA-S.}}

\newglossaryentry{Headquarters}{name={Headquarters}, description={The primary centre for CTAO governance, admin